In [1]:
%aiida
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

In [2]:
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.array import ArrayData
from aiida.orm.data.base import Int, Str, Float, Bool
from aiida.orm.data.remote import RemoteData
from aiida.work import workfunction
from aiida.work.process import WorkCalculation
from aiida.work.run import submit
from aiida_cp2k.calculations import Cp2kCalculation

from aiida.orm import Code, Computer
from aiida.orm.querybuilder import QueryBuilder

import ase
import ase.io
import numpy as np
import nglview
from copy import deepcopy
from pprint import pprint

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML

from apps.scanning_probe.stm.stm_workchain import STMWorkChain


# Select structure

In [3]:
from apps.surfaces.structure_browser import StructureBrowser

atoms = None

def on_struct_change(c):
    global atoms
    structure = struct_browser.results.value
    if structure:
        atoms = structure.get_ase()
        atoms.pbc = [1, 1, 1]
        
        unobserve_cell_text()
        cell_x_text.value = atoms.cell[0,0]
        cell_y_text.value = atoms.cell[1,1]
        cell_z_text.value = atoms.cell[2,2]
        observe_cell_text()
    
    update_view(atoms)

def on_cell_change(c):
    global atoms
    atoms.cell = np.diag(np.array([
        cell_x_text.value,
        cell_y_text.value,
        cell_z_text.value
    ]))
    atoms.center()
    update_view(atoms)
    
def observe_cell_text():
    cell_x_text.observe(on_cell_change, names='value')
    cell_y_text.observe(on_cell_change, names='value')
    cell_z_text.observe(on_cell_change, names='value')
    
def unobserve_cell_text():
    cell_x_text.unobserve(on_cell_change, names='value')
    cell_y_text.unobserve(on_cell_change, names='value')
    cell_z_text.unobserve(on_cell_change, names='value')
    
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')
viewer = nglview.NGLWidget()
clear_output()

cell_x_text = ipw.FloatText(description='cell x', value=0.0)
cell_y_text = ipw.FloatText(description='cell y', value=0.0)
cell_z_text = ipw.FloatText(description='cell z', value=0.0)

observe_cell_text()

display(ipw.VBox([struct_browser, viewer, cell_x_text, cell_y_text, cell_z_text]))

def update_view(atoms):
    
    # remove old components
    if hasattr(viewer, "component_0"):
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    
    if atoms:
        # add new component
        viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
        viewer.add_unitcell()
        viewer.center()
        
        # Orient camera to look from positive z
        cell_z = atoms.cell[2, 2]
        com = atoms.get_center_of_mass()
        def_orientation = viewer._camera_orientation
        top_z_orientation = [1.0, 0.0, 0.0, 0,
                             0.0, 1.0, 0.0, 0,
                             0.0, 0.0, -np.max([cell_z, 30.0]) , 0,
                             -com[0], -com[1], -com[2], 1]
        viewer._set_camera_orientation(top_z_orientation)
        #viewer.camera = 'orthographic'

VBox(children=(StructureBrowser(children=(VBox(children=(VBox(children=(HTML(value=u'<p>Select the date range:</p>'), HBox(children=(Text(value=u'2018-10-30', description=u'From: ', style=DescriptionStyle(description_width=u'120px')), Text(value=u'2018-11-07', description=u'To: '))), Button(description=u'Search', layout=Layout(margin=u'1em 0 0 0'), style=ButtonStyle())), layout=Layout(border=u'1px solid #fafafa', padding=u'1em')), HTML(value=u'<hr>'), HBox(children=(RadioButtons(layout=Layout(width=u'25%'), options=('all', 'uploaded', 'edited', 'calculated'), value='all'), Dropdown(description=u'Process Label', layout=Layout(width=u'50%'), options=('All', u'STMWorkChain', u'PdosWorkChain', u'SlabGeoOptWorkChain', u'NanoribbonWorkChain', u'AfmWorkChain', u'ReplicaWorkchain', u'NEBWorkchain'), style=DescriptionStyle(description_width=u'120px'), value='All'))))), HTML(value=u'<hr>'), Dropdown(layout=Layout(width=u'900px'), options=OrderedDict([('Select a Structure (9 found)', False), (u'PK: 4736 | 2018-11-06 13:10 | C48H18 | mol from slab PK4732', <StructureData: uuid: c2fc903c-3c2e-4d2d-a3cc-f475432e720e (pk: 4736)>), (u'PK: 4732 | 2018-11-06 05:38 | C48H242Au896 | ', <StructureData: uuid: 6630fe39-afd5-4ed8-b8c4-3c05a2a2ec0a (pk: 4732)>), (u'PK: 4714 | 2018-11-05 22:16 | C48H242Au896 | upload(C48H242Au896)_z4_on_au111_opt', <StructureData: uuid: 043487ac-733d-433f-9eaa-4468d3e47359 (pk: 4714)>), (u'PK: 4679 | 2018-11-04 23:01 | C28H12 | mol from slab PK4519', <StructureData: uuid: f979bd1f-6ee5-49a5-890f-6f61a804d0e7 (pk: 4679)>), (u'PK: 4663 | 2018-11-04 22:52 | C28H12 | mol from slab PK4519', <StructureData: uuid: 22179c3e-4f71-4aef-b1dd-86776ec4955c (pk: 4663)>), (u'PK: 4595 | 2018-11-04 14:00 | C28H12 | mol from slab PK4519', <StructureData: uuid: 8ceb5fd2-c7c5-467d-9e61-2e5e641414bb (pk: 4595)>), (u'PK: 4555 | 2018-11-04 00:07 | C28H12 | mol from slab PK4519', <StructureData: uuid: d9268af1-bfc1-431c-b0d7-ff9fe06ea751 (pk: 4555)>), (u'PK: 4532 | 2018-11-03 14:42 | C28H12 | mol from slab PK4519', <StructureData: uuid: c09935b5-8269-4844-9a2a-6c1662f2c058 (pk: 4532)>), (u'PK: 4519 | 2018-11-02 22:32 | C28H236Au898 | mol from slab PK4087_Au(111).slab(14,8,4)', <StructureData: uuid: 4cce670f-9e7f-4395-a925-ed154251f5bc (pk: 4519)>)]), value=False))), NGLWidget(), FloatText(value=0.0, description=u'cell x'), FloatText(value=0.0, description=u'cell y'), FloatText(value=0.0, description=u'cell z')))

# Select computer and codes

In [ ]:
qb = QueryBuilder()
qb.append(Computer, filters={'enabled': True}, project='name')
computer_names = [comp[0] for comp in qb.all()]

style = {'description_width': '120px'}
layout = {'width': '70%'}
drop_computer = ipw.Dropdown(description="Computer",
                             options=computer_names)

def comp_plugin_codes(computer_name, plugin_name):
    qb = QueryBuilder()
    qb.append(Computer, filters={'enabled': True}, project='name', tag='computer')
    qb.append(Code, project='*', has_computer='computer', filters={
        'attributes.input_plugin': plugin_name,
        'or': [{'extras': {'!has_key': 'hidden'}}, {'extras.hidden': False}]
    })
    qb.order_by({Code: {'id': 'desc'}})
    codes = qb.all()
    sel_codes = []
    for code in codes:
        if code[0] == computer_name:
            sel_codes.append(code[1])
    return sel_codes

def on_computer_change(c):
    global cp2k_codes, stm_codes
    cp2k_codes = comp_plugin_codes(drop_computer.value, 'cp2k')
    stm_codes = comp_plugin_codes(drop_computer.value, 'spm.stm')
    
    drop_cp2k.options = [c.label for c in cp2k_codes]
    drop_stm.options = [c.label for c in stm_codes]
    
    
drop_computer.observe(on_computer_change)

drop_cp2k = ipw.Dropdown(description="Cp2k code")

drop_stm = ipw.Dropdown(description="STM code")

on_computer_change(0)

elpa_check = ipw.Checkbox(
    value=True,
    description='use ELPA',
    disabled=False
)

display(drop_computer, drop_cp2k, drop_stm, elpa_check)

# Scanning tunnelling microscopy parameters

In [ ]:
style = {'description_width': '140px'}
layout = {'width': '50%'}
layout_small = {'width': '25%'}

elim_float_slider = ipw.FloatRangeSlider(
    value=[-2.0, 2.0],
    min=-4.0,
    max=4.0,
    step=0.1,
    description='Emin, Emax (eV):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    style=style, layout=layout)

const_height_text = ipw.Text(description='Const. H (ang)',
                              value='3.0 5.0',
                              style=style, layout=layout)

const_current_text = ipw.Text(description='Const. cur. (isoval)',
                              value='1e-8 1e-7 1e-6',
                              style=style, layout=layout)

extrap_plane_floattext = ipw.BoundedFloatText(
                        description='Extrap plane (ang)',
                        min=1.0,
                        max=10.0,
                        step=0.1,
                        value=3.0,
                        style=style, layout=layout_small)

de_floattext = ipw.BoundedFloatText(
                        description='dE (eV)',
                        min=0.01,
                        max=0.10,
                        step=0.01,
                        value=0.05,
                        style=style, layout=layout_small)

fwhm_floattext = ipw.BoundedFloatText(
                        description='FWHM (eV)',
                        min=0.05,
                        max=0.50,
                        step=0.01,
                        value=0.10,
                        style=style, layout=layout_small)

display(elim_float_slider, const_height_text, const_current_text, extrap_plane_floattext, de_floattext, fwhm_floattext)

In [ ]:
def find_struct_wf(structure):
    qb = QueryBuilder()
    qb.append(Computer, filters={'enabled': True})
    selected_comp = qb.all()[drop_computer.index][0]
    
    # check stm
    extras = structure.get_extras()
    for ex_k in extras.keys():
        if ex_k.startswith(('stm', 'pdos', 'afm')):
            spm_workchain = load_node(extras[ex_k])
            cp2k_scf_calc = spm_workchain.get_outputs()[0]
            if cp2k_scf_calc.get_computer().hostname == selected_comp.hostname:
                wfn_path = cp2k_scf_calc.out.remote_folder.get_remote_path() + "/aiida-RESTART.wfn"
                # check if it exists
                file_exists = ! ssh {selected_comp.hostname} "if [ -f {wfn_path} ]; then echo 1 ; else echo 0 ; fi"
                if file_exists[0]:
                    print("Found .wfn from %s"%ex_k)
                    return wfn_path
    
    # check geo opt
    geo_opt_calc = structure.get_inputs()[0]
    geo_comp = geo_opt_calc.get_computer()
    if geo_comp.hostname == selected_comp.hostname:
        wfn_path = geo_opt_calc.out.remote_folder.get_remote_path() + "/aiida-RESTART.wfn"
        # check if it exists
        file_exists = ! ssh {selected_comp.hostname} "if [ -f {wfn_path} ]; then echo 1 ; else echo 0 ; fi"
        if file_exists[0]:
            print("Found .wfn from geo_opt")
            return wfn_path
    return ""

def on_submit(b):
    with submit_out:
        clear_output()
        if not struct_browser.results.value:
            print("Please select a structure.")
            return
        if not drop_computer.value:
            print("Please select a computer.")
            return
        
        extrap_plane = extrap_plane_floattext.value
        parent_dir = "parent_calc_folder/"

        stm_params = ParameterData(dict={
            '--cp2k_input_file':    parent_dir+'aiida.inp',
            '--basis_set_file':     parent_dir+'BASIS_MOLOPT',
            '--xyz_file':           parent_dir+'geom.xyz',
            '--wfn_file':           parent_dir+'aiida-RESTART.wfn',
            '--hartree_file':       parent_dir+'aiida-HART-v_hartree-1_0.cube',
            '--output_file':        'stm.npz',
            '--emin':               str(elim_float_slider.value[0]),
            '--emax':               str(elim_float_slider.value[1]),
            '--eval_region':        ['G', 'G', 'G', 'G', 'n-2.0_C', 'p%.1f'%extrap_plane],
            '--dx':                 '0.15',
            '--eval_cutoff':        '14.0',
            '--extrap_extent':      '5.0',
            '--heights':            const_height_text.value.split(),
            '--isovalues':          const_current_text.value.split(),
            '--de':                 str(de_floattext.value),
            '--fwhm':               str(fwhm_floattext.value),
        })
        
        cp2k_code = cp2k_codes[drop_cp2k.index]
        stm_code = stm_codes[drop_stm.index]
        
        struct = struct_browser.results.value
        
        cell = ArrayData()
        cell.set_array('cell', np.diag(atoms.cell))
        
       ## Try to access the restart-wfn file ##
        try:
            wfn_file_path = find_struct_wf(struct)
        except:
            wfn_file_path = ""
        if wfn_file_path == "":
            print("Didn't find any accessible .wfn file.")
    
        
        outputs = submit(STMWorkChain,
                 cp2k_code=cp2k_code,
                 structure=struct,
                 cell=cell,
                 wfn_file_path=Str(wfn_file_path),
                 elpa_switch=Bool(elpa_check.value),
                 stm_code=stm_code,
                 stm_params=stm_params
                )
                
        print(outputs)

btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)